In [42]:
import dill as pickle
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix as cm
from sklearn.metrics import plot_confusion_matrix
from tensorflow.keras.models import load_model

In [43]:
def calculate_metrics_per_class(confusion_matrix):
    """
    Calculate sensitivity and specificity for each class from a confusion matrix.

    Parameters:
    - confusion_matrix: numpy array, shape (n_classes, n_classes)
        The confusion matrix.

    Returns:
    - sensitivities: numpy array, shape (n_classes,)
        Sensitivity (True Positive Rate or Recall) for each class.
    - specificities: numpy array, shape (n_classes,)
        Specificity (True Negative Rate) for each class.
    """
    n_classes = confusion_matrix.shape[0]
    sensitivities = np.zeros(n_classes)
    specificities = np.zeros(n_classes)

    for i in range(n_classes):
        true_positive = confusion_matrix[i, i]
        false_negative = np.sum(confusion_matrix[i, :]) - true_positive
        false_positive = np.sum(confusion_matrix[:, i]) - true_positive
        true_negative = np.sum(confusion_matrix) - (true_positive + false_negative + false_positive)

        # Sensitivity (True Positive Rate or Recall) for each class
        sensitivities[i] = true_positive / (true_positive + false_negative)

        # Specificity (True Negative Rate) for each class
        specificities[i] = true_negative / (true_negative + false_positive)

    return sensitivities, specificities

# Load the Cleaned test data

In [44]:
device_test_data = pd.read_csv("Cleaned_test.csv")
device_test_copy = device_test_data.copy()
device_test_copy.head()

,battery_power,blue,clock_speed,dual_sim,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,ram,talk_time,touch_screen,wifi,px_total,sc_diagonal,price_range
0,0.413462,-0.992494,1.566979,0.978631,-1.042594,0.988532,-1.393208,-1.137703,1.520803,-0.315058,-1.254295,1.097922,0.996490,0.989508,-0.766539,0.596349,0
1,-0.455647,-0.992494,-0.395503,-1.021836,-1.042594,0.602509,0.686160,0.020820,-1.535056,-0.810148,1.461466,0.182757,0.996490,0.989508,-1.090228,-0.581328,3
2,-1.536345,-0.992494,0.585738,0.978631,-1.042594,0.823093,-0.700085,-0.713853,1.084252,0.345063,0.818939,0.731856,-1.003522,-1.010604,-0.918799,0.114064,2
3,-0.535277,1.007562,1.689635,-1.021836,0.959146,-0.886438,0.339599,-0.600827,1.520803,1.500275,-0.300184,-0.183309,-1.003522,-1.010604,-0.655647,0.681288,1
4,1.344000,1.007562,0.708393,-1.021836,-1.042594,1.484848,-1.393208,-1.392013,0.211150,0.840154,0.938779,-1.098473,0.996490,0.989508,2.287569,0.607799,3


In [45]:
X = device_test_copy.drop(['price_range'], axis=1)
Y = device_test_copy['price_range']


# Load the Test data to be predicted

In [67]:
# Load your DataFrame from the CSV file
device_pred = pd.read_csv("Cleaned_APP_test.csv")

# Make a copy of the DataFrame
device_pred_copy = device_pred.copy()

# Get a random sample of 10 data points
random_sample = device_pred_copy.sample(n=10)  # Set random_state for reproducibility
random_sample = random_sample.drop(['id'], axis=1)
# Display the random sample
print(random_sample)

     battery_power  blue  clock_speed  dual_sim  four_g  int_memory  m_dep  \
143           1466     0          2.4         1       1          57    0.9   
232           1580     0          2.8         1       0          45    1.0   
484           1546     1          2.7         0       0          55    0.2   
157           1496     1          1.3         1       0          42    0.9   
780           1986     0          0.8         0       1          13    0.2   
134           1140     0          1.6         1       1          56    0.3   
165            796     1          2.3         1       1          36    0.3   
947           1347     0          2.1         1       0          44    0.8   
675           1906     0          0.9         1       0          14    0.5   
492            657     0          1.6         1       0          35    0.6   

     mobile_wt  n_cores  pc   ram  talk_time  touch_screen  wifi  px_total  \
143        155        6  18  2340          4             1     

# Load the ANN pre-trained mode

In [69]:
# To load the model back:
ANN_model = load_model('ANN_model')

#### Test the loaded model

In [70]:
# Predict Using ANN model
y_ANN_pred = np.argmax(ANN_model.predict(X), axis=1)
# Get the confusion matrix
conf_ANN_mat = cm(y_ANN_pred, Y)
print("Comfusion Matrix:")
print(conf_ANN_mat)

sensitivities, specificities = calculate_metrics_per_class(conf_ANN_mat)

# Print or use the metrics per class
for i in range(len(sensitivities)):
    print(f"Class {i + 1}: Sensitivity = {sensitivities[i]}, Specificity = {specificities[i]}")

# Calculate the accuracy: 
accuracy=np.trace(conf_ANN_mat)/np.sum(conf_ANN_mat)
print("The accuracy is: ", accuracy)


Comfusion Matrix:
[[ 95   0   0   0]
 [  5  98   0   0]
 [  0   1 100   1]
 [  0   0   0  99]]
Class 1: Sensitivity = 1.0, Specificity = 0.9835526315789473
Class 2: Sensitivity = 0.9514563106796117, Specificity = 0.9966216216216216
Class 3: Sensitivity = 0.9803921568627451, Specificity = 1.0
Class 4: Sensitivity = 1.0, Specificity = 0.9966666666666667
The accuracy is:  0.9824561403508771


# Predict 10 data point from the test data

In [71]:
# Predict Using ANN model 10 sample data
y_pred_APP = np.argmax(ANN_model.predict(random_sample), axis=1)

In [72]:
# Create a DataFrame with the predictions
predictions_df = pd.DataFrame({"Prediction": y_pred_APP})

# Print the DataFrame
print(predictions_df.to_string(index=False))

 Prediction
          3
          3
          3
          3
          3
          3
          3
          3
          3
          3


In [51]:
import tensorflow as tf
row_data = X.iloc[5]

# Convert row data to a NumPy array
row_array = row_data.values

# Convert NumPy array to a TensorFlow tensor
row_tensor = tf.convert_to_tensor(row_array, dtype=tf.float32)

row_tensor


<tf.Tensor: shape=(16,), dtype=float32, numpy=
array([ 0.46351555, -0.9924943 , -1.1314335 ,  0.9786311 , -1.042594  ,
       -0.2798302 , -0.35352388, -0.3747733 ,  0.21114962, -0.31505767,
       -1.0441141 , -0.73240733, -1.003522  , -1.0106037 , -0.8868179 ,
        0.4196069 ], dtype=float32)>

In [52]:

import requests 
def testpredict():
    # Test case 1: Valid input features
    response = requests.post("http://localhost:8080/predict", json=row_tensor)
    print(response)

if __name__ == "__main__":
    testpredict()

TypeError: Object of type 'EagerTensor' is not JSON serializable